In [ ]:
!pip list | grep numpy

numpy                              2.0.2


In [ ]:
!pip list | grep pandas

geopandas                          1.0.1
pandas                             2.2.2
pandas-datareader                  0.10.0
pandas-gbq                         0.28.0
pandas-stubs                       2.2.2.240909
sklearn-pandas                     2.2.0


In [ ]:
!pip list | grep matplotlib

matplotlib                         3.10.0
matplotlib-inline                  0.1.7
matplotlib-venn                    1.1.2


In [ ]:
!pip list | grep seaborn

seaborn                            0.13.2


In [ ]:
!pip list | grep sklearn

sklearn-compat                     0.1.3
sklearn-pandas                     2.2.0


In [ ]:
# Data, Arrays, Linear Algebra
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Add for better Jupyter Notebook compatibility
%matplotlib inline

# Modeling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Load Data
# Source: Obtained from the "Python for Data Science and Machine Learning Bootcamp"
# Reference: https://www.udemy.com/certificate/UC-a9c3a1ee-f780-447f-8bdf-5a9c2afb8dd0/
from google.colab import files
uploaded = files.upload()